<a href="https://colab.research.google.com/github/varshasing/rl-blackjack-agent/blob/main/Reinforcement_Learning_Blackjack_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning to play Blackjack

Implementation by Varsha Singh @ Boston University



Blackjack is a card game played against a "dealer". The full rules, including various betting rules we will not use, are available online here: https://bicyclecards.com/how-to-play/blackjack/

The object of the game is to collect a hand of cards whose point total is as close to 21 as possible *WITHOUT* going over. If your point total is over 21, then the dealer automatically wins. A hand of cards is scored as follows:

Each card is worth a certain number of points. For a card specified by a `(value, suit)` tuple,
the number of points it is worth is completely determined by the  `value` term in the tuple (`suit` is irrelevant).

* If `value` is a number "2" through "10", then the card is worth that number of  points.
* If `value` is "Jack", "Queen" or "King", then the card is worth 10 points.
* If `value` is "Ace", then the card is worth EITHER 1 point or 11 points - the player may choose.

The total score of a hand is the total number of points for all cards in the hand, counting each Ace in whichever manner (either 11 or 1) the player chooses.

At the start of the game, you are dealt two cards, and the dealer is dealt  1 card that is visible to you. You may take one of  two actions: a "HIT" or  a "STAY". If you take the "HIT" action, you will be given another card. Getting another card will increase your score, but might also make your score go over 21. If you take the "STAY" action, this  indicates that you  are satisfied  with  your current point total (i.e. you suspect that taking another card might make your score go over  21, or that the dealer will not able to beat your  score without  going over 21 themselves). Not that you may adjust your decision to count an Ace as 1 point  or 11 points at  any  time. So, for  example, if you hand consists of a 9 of clubs and an Ace of spades, you might count the Ace as 11 points for a total of 20 points. However, if you then take the "HIT" action (which is very inadvisable in this case!), you might  be dealt an 8 of hearts, in which case your  total would be  either 28 (counting the Ace as 11)  or 18  (counting  the Ace as 1). So, you could now count the Ace as 1 point and choose the  "STAY" action, to indicate that you are no longer interested in taking another car.

If you take the "HIT" action, then you are dealt a card and again have the option to either take a "HIT" or "STAY" action. You may continue taking the "HIT" action and getting cards until your total is over 21 no matter how you choose to count the Aces. If this happens, then your hand is called "bust" and you automatically lose. So, you should try to choose  the "STAY" action before this occurs.

If you take the "STAY" action, then the dealer will start playing their hand.  The dealer is not allowed to strategize: the  dealer will continually take  the  "HIT" action until their hand is worth 17 points or more. Once that happens, the dealer takes the "STAY" action. The dealer must count Aces as  11 points unless counting the Ace this  way would  cause the  dealer's score to go  over 21.

Outcome of the game:

If you go bust, you automatically lose. If you do  not go bust,  but the dealer  does, then you win. If neither of you go bust, then the person with the  highest score  wins. Otherwise, it is a tie.


Note that the player gets to see the  first card that  the dealer is dealt, but NOT any of the other cards that the dealer will take when making their decisions.


### Reward Structure
To fit this game into the reinforcement learning framework, we need to decide on what rewards to give the  player. We give rewards in the following way:

* If the player takes an action that results in a loss (either "STAY" and then losing after the dealer finishes playing, or "HIT" and going bust), then the  reward is -1.

* If the player takes an action that results in a win (i.e. a "STAY" followed by the dealer either going bust or  having a smaller score than the  player), then the reward is 1.

* If the player takes an action that results in a tie (i.e. a "STAY" followed by the dealer achieving the same score as the player), then the reward is 0.

* If the player takes  an action that does  not result in  the game ending (that is, a "HIT" with the possibility of taking another "HIT"), then the reward is 0.

## Action Structure

The  set of actions  available to the  player  is always either both "HIT" and "STAY", or nothing at all (which happens after taking the "STAY" action, or going  bust), both of which result in entering a terminal state, as the dealer will play out their hand and the player will take no more actions.


Blackjack simulator, converting 'full state' of the blackjack game into a 'minimal state' that contains just the critical information needed for a reinforcement learning agent to play the game optimally.



In [ ]:
# Two functions for running a blackjack game

def draw_card():
  '''
  returns a randomly selected (value, suit) tuple representing a random card.
  Assumes cards are drawn from infinitely many decks shuffled together (so
  your agent cannot gain an advantage by counting cards in this game).
  '''
  value = random.choice(['Ace'] + [str(i) for i in range(2,11)] + ['Jack', 'Queen', 'King'])
  suit = random.choice(['Clubs', 'Hearts', 'Spades', 'Diamonds'])

  return (value, suit)

def hand_to_str(hand):
  '''
  Returns a string containing a human-readable description of a blackjack hand.
  Note this function returns a string, it does not call `print`.
  '''
  def card_to_str(c):
    value, suit = c
    return value + ' of ' + suit
  return ', '.join([card_to_str(c) for c in hand])

In [ ]:
def score_hand(hand):
  '''
  returns the best possible score for a given hand.

  You can read the full scoring rules below, and also at this link https://bicyclecards.com/how-to-play/blackjack/ under "Card Values/scoring".

  The rules for scoring are:
  Each card is worth a certain number of points. For a card specified by a (value, suit) tuple,
  the number of points it is worth is completely determined by the  "value" term in the tuple (suit is irrelevant).
  If the value term is a number 2 through 10, then the card is worth that number of  points.
  If the value is "Jack", "Queen" or "King", then the card is worth 10 points.
  If the value is "Ace", then the card is worth EITHER 1 point or 11 points - the player may choose.

  The total score of a hand is the total number of points for all cards in the hand, counting each Ace in whatever manner
  the player chooses.
  The objective of the game is to  have as high as score as possible WITHOUT going over 21. So a score of 21 is a "perfect" score,
  a score of 20 is a very good score, and a score of 22 is very bad score (if you go over 21 you automatically lose).

  arguments:
    hand: a list of (value, suit) pairs specifying the hand of the player.
        both "value" and "suit" are represented by strings.
  return two values:
    (score, eleven_valued_aces)

    score: a non-negative integer representing either:
      1. The highest score possible with this hand that is less than or equal to 21. Multiple scores  might be  possible  if the  hand
        contains one or more  Aces. You  must find the  largest score that is less than or equal to 21 if such a thing exists.
      2. If it is impossible for this hand to score less than or equal to 21 (the hand is called "bust" in this case),
        then return the lowest possible score for the hand (which must necessarily be >21).

    eleven_valued_aces: the number of aces in the hand that were scored at 11 points rather than 1 point
      in order to achieve the score provided in the first return value (these are also called "soft" aces).
    '''

  # add all non-aces and for every ace, add 11 if under 21 : add 1
  aces = 0
  eleven_aces = 0
  score = 0
  for card in hand:
    if card[0] == 'Ace':
      aces += 1
    elif card[0] == 'Jack' or card[0] == 'Queen' or card[0] == 'King':
      score += 10
    else:
      score += int(card[0])
  while aces > 0:
    if score + 11 <= 21:
      score += 11
      eleven_aces += 1
    else:
      score += 1
    aces -= 1
  return score, eleven_aces

In [ ]:
test_cases = {
  (('Ace', 'Clubs'),): (11, 1),
  (('Ace', 'Hearts'), ('Ace', 'Spades'),): (12, 1),
  (('8', 'Clubs'), ('King', 'Diamonds'), ('9', 'Spades'),): (27, 0),
  (('Ace', 'Hearts'), ('7', 'Diamonds'), ('Ace', 'Clubs'), ('King', 'Diamonds'),): (19, 0),
  (('10', 'Spades'),): (10, 0),
  (('King', 'Spades'), ('Ace', 'Clubs'),): (21, 1),
  (('Ace', 'Hearts'), ('7', 'Diamonds'), ('10', 'Diamonds'),): (18, 0),
  (('10', 'Clubs'), ('4', 'Hearts'), ('Ace', 'Spades'), ('8', 'Spades'),): (23, 0),
  (('4', 'Clubs'), ('7', 'Hearts'), ('Ace', 'Spades'), ('2', 'Clubs'), ('King', 'Diamonds'),): (24, 0),
  (('2', 'Hearts'), ('5', 'Hearts'), ('2', 'Hearts'), ('3', 'Hearts'), ('3', 'Diamonds'), ('King', 'Spades'),): (25, 0),
  (('10', 'Clubs'),): (10, 0),
  (('King', 'Hearts'), ('Ace', 'Spades'),): (21, 1),
  (('4', 'Spades'), ('5', 'Diamonds'), ('3', 'Hearts'),): (12, 0),
  (('4', 'Diamonds'), ('8', 'Hearts'), ('4', 'Spades'), ('6', 'Diamonds'),): (22, 0),
  (('Jack', 'Spades'),): (10, 0),
  (('10', 'Diamonds'), ('7', 'Clubs'),): (17, 0),
  (('Jack', 'Hearts'), ('6', 'Clubs'), ('6', 'Spades'),): (22, 0),
  (('2', 'Diamonds'), ('Ace', 'Spades'), ('8', 'Spades'), ('7', 'Spades'),): (18, 0),
  (('King', 'Clubs'),): (10, 0),
  (('Ace', 'Hearts'), ('Jack', 'Diamonds'),): (21, 1),
  (('4', 'Clubs'), ('2', 'Diamonds'), ('4', 'Spades'),): (10, 0),
  (('King', 'Spades'), ('10', 'Hearts'), ('2', 'Clubs'), ('5', 'Spades'),): (27, 0),
  (('Queen', 'Hearts'),): (10, 0),
}

def test_score_hand():
  for hand, true_output in test_cases.items():
    output = score_hand(list(hand))
    assert output == true_output, f"incorrect output on test case: {hand}. Expected output: {true_output}, computed output: {output}"
  print("all tests pass :)")


In [ ]:
# test scoring function
test_score_hand()

all tests pass :)


The following cell contains the blackjack game code.

In [ ]:
class BlackjackSimulator:
  '''
  This class simulates a (slightly simplified) version of the card game Blackjack.

  The player's current hand is accessible via the `player_hand` attribute.
  The dealers' current hand is accessible via the `dealer_hand` attribute.

  Both hands are stored as lists of cards, where each card is a tuple (value, suit) as
  returned by `draw_card()`

  The actions currently available to the player are accessible via `available_actions` attribute.
  The last reward earned by the player is accessible via the `reward` attribute.

  Check the docstrings of the individual methods to see what they do.
  In our reference solution, we access the `player_hand` and `dealer_hand` attribute
  in the function `get_state`. We do not use any other internals of BlackjackSimulator objects.
  However, if you find it useful to use some internal methods or attributes, you may do so.

  See the rules of blackjack here: https://bicyclecards.com/how-to-play/blackjack/

  We do not play with "naturals" here as they do not require any decision making.
  This actually makes the odds a little worse than they should be, so your agent
  would actually do slightly better in a casino than it will in this simulator.

  It still wouldn't beat the house though. Gambling is dangerous, and we do not endorse it!

  If you find any other discrepencies in the official rules and the game as implemented
  in this simulator, please write your code to do well on this simulator, not the official rules :)
  '''

  def __init__(self):
    # the player starts with two cards
    self.player_hand = [draw_card(), draw_card()]

    # the dealer starts with one visible card
    self.dealer_hand = [draw_card()]

    self.available_actions = ['HIT', 'STAY']

    # this will hold the reward of the current state of the game. It will always be
    # either -1, (loss), 0 (game in progress, or tie), or 1 (win)
    self.reward = 0


  def is_game_over(self):
    '''
    returns True if the game is over.
    At this point, self.reward will hold the result of the game
    (0 for tie, 1 for player win, -1 for player loss).
    '''
    return len(self.available_actions) == 0

  def player_hand_str(self):
    '''
    returns a string representation of the player hand (it does not actually
    call the `print` method).
    '''
    return hand_to_str(self.player_hand)

  def dealer_hand_str(self):
    '''
    returns a string representation of the dealer hand (it does not actually
    call the `print` method).
    '''
    return hand_to_str(self.dealer_hand)

  def get_reward(self):
    '''
    get reward from last action taken.
    '''
    self.reward = self._get_reward()
    return self.reward

  def _get_reward(self):
    '''
    helper function for self.get_reward: this one has all the actual logic.
    '''
    if len(self.available_actions) != 0:
      # we are still playing - you might yet win!
      return 0

    dealer_score = self.dealer_score()
    player_score = self.player_score()
    if player_score==dealer_score:
      # this is the only way to tie.
      return 0

    # we must have a winner.
    if player_score > 21:
      # dealer wins when player busts (even if the dealer also busts)
      return -1
    if dealer_score > 21:
      # dealer bust, player did not bust.
      return 1
    if player_score > dealer_score:
      return 1
    return -1

  def player_score(self):
    return score_hand(self.player_hand)[0]

  def dealer_score(self):
    return score_hand(self.dealer_hand)[0]

  def take_action(self, action: str):
    '''
    plays one turn of blackjack.

    arguments:
      action: a string value, must be an element of self.available_actions

    returns:
      a reward for this action (integer).
    '''

    assert action in self.available_actions, f"Attempted action: {action}, but the available actions are: {self.available_actions}. The current hand is: {self.player_hand()}"


    if action == 'STAY':
      self.available_actions = []
      # once the player STAYs, the dealer will play out its hand.
      while self.dealer_score() < 17:
        # dealer hits until score is 17 or bigger
        self.dealer_hand.append(draw_card())
    else:
      self.player_hand.append(draw_card())
      if self.player_score() > 21:
        # BUST!
        self.available_actions = []


    return self.get_reward()


In [ ]:
## This cell contains some helper functions that print the current state of the
# blackjack game in a human-readable way, as well as allow you to test the blackjack
# simulator by playing with it yourself.


def print_status(bs):
  print(f"Current Total: {bs.player_score()}\nCurrent hand: {bs.player_hand_str()}")
  print(f"Dealer Total: {bs.dealer_score()}\nDealer hand: {bs.dealer_hand_str()}")
  if len(bs.available_actions) == 0:
    if bs.get_reward() == 0:
      print(f"It's a tie!")
    elif bs.get_reward() == 1:
      print(f"Player Wins!")
    elif bs.get_reward() == -1:
      print(f"Player loses!")
    return False
  else:
    return True

def play_blackjack():
  bs = BlackjackSimulator()

  def get_action():
    action = None
    options = bs.available_actions
    while action not in options:
      if  action is not None:
        print(f"{action} is not an allowed action. Please select again.")
      print("Available Actions:")
      for option in options:
        print(f"* {option}")
      action = input("Your selection: ").upper()
    return action

  print_status(bs)
  while not bs.is_game_over():
    action = get_action()
    bs.take_action(action)
    print_status(bs)


If you want, run following cell to play blackjack yourself.

In [ ]:
play_blackjack()

Current Total: 14
Current hand: 3 of Clubs, Ace of Spades
Dealer Total: 10
Dealer hand: Queen of Spades
Available Actions:
* HIT
* STAY
Your selection: HIT
Current Total: 15
Current hand: 3 of Clubs, Ace of Spades, Ace of Clubs
Dealer Total: 10
Dealer hand: Queen of Spades
Available Actions:
* HIT
* STAY
Your selection: HIT
Current Total: 19
Current hand: 3 of Clubs, Ace of Spades, Ace of Clubs, 4 of Hearts
Dealer Total: 10
Dealer hand: Queen of Spades
Available Actions:
* HIT
* STAY
Your selection: STAY
Current Total: 19
Current hand: 3 of Clubs, Ace of Spades, Ace of Clubs, 4 of Hearts
Dealer Total: 26
Dealer hand: Queen of Spades, 6 of Clubs, 10 of Spades
Player Wins!


`get_state` captures the minimal information needed by an agent to make an optimal decision about what action to take. As specified in the docstring, the input should be a BlackjackSimulator object as defined above.

In [ ]:
def get_state(sim: BlackjackSimulator):
  '''
  find the score and number of aces values at 11 points rather than 1 point in both the player and dealer's hands (aka "soft aces").

  arguments:
    sim: a BlackjackSimulator object.

  returns:
     a tuple (player_score, player_eleven_valued_aces, dealer_score, dealer_eleven_valued_aces)
  '''

  # extract a tuple (player_score, player_eleven_valued_aces, dealer_score, dealer_eleven_valued_aces)
  player_score, player_eleven_valued_aces = score_hand(sim.player_hand)
  dealer_score, dealer_eleven_valued_aces = score_hand(sim.dealer_hand)
  return player_score, player_eleven_valued_aces, dealer_score, dealer_eleven_valued_aces

In [ ]:
# Argument checking functions that are useful for making sure implementation
# of Q-learning is passing the correct values around.

def assert_valid_action(action):
  assert action in ['HIT', 'STAY'], f"invalid action: {action}"

def assert_valid_state(state):
  assert type(state) == tuple, f"state must be a tuple, but was: {state}"

  p_score, p_aces, d_score, d_aces = state
  assert type(p_score) == int and type(p_aces) == int and type(d_score) == int and type(d_aces) == int, f"state has invalid types: {state}"
  assert p_score>=4 and p_aces>=0 and p_score<=31 and d_score>=2 and d_aces>=0 and d_score<=27, f"state has impossible values: {state}"



The output of `get_state` is sufficient for designing an optimal agent because using the full state of 52 different cards (up to) requires a lot more space and time for computations. When playing the game, I thought moreso about how strong the dealers hand is and compared it to mine, as well as the amount of flexibility I had with aces. It wasn't really a consideration of mine to think about the different permutations of amounting to a score, as there are 4 different ways of having a card be a value of 2, So, I believe that the neccessary information for an optimal agent shouldn't worry about the different suites and rather worry about the numerical scores and how much flexibility they have with their aces, as they can be two-valued.

In [ ]:
class Q_Function:
  '''
  class for holding a Q function.
  This class is essentially a manual implementation of a DefaultDict with
  default value zero and a little type checking on top to help catch some bugs
  that might occur when you use it.
  '''
  def __init__(self):
    self.Q = dict()

  def set_value(self, state, action, value):
    '''
    sets the estimated Q(s, a) to be a given value.

    arguments:
      state: the state (a tuple as returned  by get_state)
      action:  a string.
      value: a float.

    returns
      None

    This function updates the Q function so that Q(state, action) = value.
    '''

    assert_valid_state(state)
    assert_valid_action(action)
    assert value <=1.00001 and value >= -1.00001, f"Q learning should never set a Q value of: {value}!"
    if state not in self.Q:
      self.Q[state] = {}
    self.Q[state][action] = value

  def get_value(self, state, action):
    '''
    Return Q(s,a): gets the estimated Q value for a given state, action pair.
    '''
    assert_valid_state(state)
    assert_valid_action(action)
    if not self.seen(state, action):
      return 0.0
    return self.Q[state][action]


  ## the following functions are  not needed, but might be useful
  ## in debugging
  def get_seen_actions_for_state(self, state):
    '''
    for a given state, return a list of all actions that have been "seen"
    in the  sense that the agent has taken this action from this state.
    '''
    assert_valid_state(state)
    if state not in self.Q:
      return []
    return self.Q[state].keys()

  def seen(self, state, action):
    '''
    Return True if the given  state, action pair has been visited by the agent before.
    False otherwise.
    '''
    assert_valid_state(state)
    assert_valid_action(action)
    if state not in self.Q:
      return False
    if action not in self.Q[state]:
      return False
    return True


## Implement Q-Learning.

### Notes on Q Learning.

Q-learning allows us to train a policy on-the-fly while playing many games of Blackjack. We maintain "Q-function": $Q(s,a)$. Eventually $Q(s,a)$ should equal:
$$
Q(s,a) = r(s,a) + \sum_{s'} P(s'|s,a) V_{\pi_\star}(s')
$$
That is, $Q(s,a)$ records the total reward we can expect to obtain if we (1) start in state $s$, and then (2) take action $a$, and finally (3) follow the optimal policy $\pi_\star$ for all subsequent actions. If we knew this $Q$ function, exactly, then it would hold that $\pi_\star(s) = \text{argmax}_a Q(s,a)$.

Furthermore, the correct $Q$ function should satisfy the following identities:
$$
\begin{align*}
V_{\pi_\star}(s) &= \max_a Q(s,a)\\
Q(s,a) &= r(s,a) + \sum_{s'} P(s'|s,a) \max_a Q(s', a)
\end{align*}
$$

Q-learning does the following: we maintain an estimate of $Q(s,a)$ for all states/actions $s,a$. (i.e. initialized as $Q(s,a)=0$ to begin). Our estimate will be *incorrect* throughout the learning process, but will slowly converge to the true value. Now, when in state $s_t$, the learner will:

1. Take action $a_t=\text{argmax}_{a} Q(s_t,a)$.
2. See reward $r_t=r(s,a)$, transition to state $s_{t+1}$.
3. Update the value of $Q(s_t,a_t)$ via the formula:
$$
Q(s_t, a_t)\leftarrow Q(s_t, a_t) + \eta \cdot (\max_a Q(s_{t+1}, a) + r_t - Q(s_t, a_t))
$$
where $\eta$ is some provided learning rate.

Intuitively, in step (1), we take an action "assuming" that our current estimate $Q$ is the "correct" $Q$ function (which is not actually true, but might become more true over time). Then, in step 3, we use our observed reward and state transition to update our estimate of $Q(s_t,a_t)$ to be hopefully more correct.


In [ ]:

def max_q_for_state(Q, state, actions):
  '''
  finds the best action to take in a given state according to the current estimated Q function.

  arguments:
  Q:  a Q_function object (see  definition above)
  state: a tuple (player_score, player_aces, dealer_score, dealer_aces) as returned by get_state.
  actions: available actions in this state.

  returns:
  (max_value, best_action)

  max_value: the maximum Q function value associated with any action in this state.
  best_action: the  action that gives the maximum Q function value from this state.
  '''

  # return max_value and best_action
  max_value = -np.inf
  best_action = None

  for action in actions:
    value = Q.get_value(state, action)
    if value > max_value:
      max_value = value
      best_action = action
  return max_value, best_action
  raise NotImplementedError


class Q_Learner:
  '''
  A Q-learning class.
  '''

  def __init__(self, eta):
    '''
    eta: the learning rate for Q learning.
    '''
    self.Q = Q_Function()
    self.eta = eta

  def update(self, prev_state, prev_action, sim, reward):
    '''
    Updates the estimated Q function (self.Q) after taking
    action `prev_action` in state `prev_state` and seeing reward  `reward`.
    `sim` holds the BlackjackSimulator object that can  be used  to determine the
    current state after taking action `prev_action`.

    arguments:
      prev_state: the previous state of the game (a tuple as returned  by get_state)
      prev_action: the action previously taken by the player.
      sim: a BlackjackSimulator object.
      reward: the reward obtained by taking prev_action in prev_state.

    returns:
      None.

    This function has no return value, but should update self.Q using the Q-learning
    update rule.
    '''

    assert_valid_state(prev_state)
    assert_valid_action(prev_action)

    current_state = get_state(sim)
    max_value, best_action = max_q_for_state(self.Q, current_state, ["HIT", "STAY"])

    current_q = self.Q.get_value(prev_state, prev_action)

    self.Q.set_value(prev_state, prev_action, current_q + self.eta * (reward + max_value - current_q))


  def get_action(self, sim):
    '''
    find the action this agent will take given the current game state as
    specified by the BlackjackSimulator object `sim` and the current estimated
    Q function stored in `self.Q`.

    arguments:
      sim: a BlackjackSimulator object containing the current game state.

    returns:
      action: a string representing the action to take next (i.e. either "HIT" or "STAY")
    '''

    state = get_state(sim)
    actions = ["HIT", "STAY"]

    max_value, best_action = max_q_for_state(self.Q, state, actions)
    return best_action
    raise NotImplementedError



In [ ]:
def run_Q_learning(iter_count, eta):
  policy = Q_Learner(eta)
  pbar = tqdm(range(iter_count))
  for i in pbar:
    sim = BlackjackSimulator()
    while not sim.is_game_over():
      state = get_state(sim)
      action = policy.get_action(sim)
      reward = sim.take_action(action)
      policy.update(state, action, sim, reward)
  return policy

In [ ]:
def run_policy_verbose(learner, verbose_level=1):
  '''
  Run a learned policy on a single game of blackjack.
  Prints out the states and actions taken during the game.
  You may control the amount of information printed by increasing `verbose_level`.
  '''
  sim = BlackjackSimulator()
  print_status(sim)
  while not sim.is_game_over():
    action = learner.get_action(sim)
    state = get_state(sim)

    print(f"Learner takes action: {action}")

    if verbose_level > 0:
      seen_actions = learner.Q.get_seen_actions_for_state(state)
      print(f"Actions tested in training: {seen_actions}")
      for seen_action in seen_actions:
        print(f"Q value for action {seen_action}: {learner.Q.get_value(state, seen_action)}")

    sim.take_action(action)

    print_status(sim)

  return sim.reward



In [ ]:
# This cell and the next one are to test the learner with a small number
# of iterations
small_policy = run_Q_learning(100, 0.1)

100%|██████████| 100/100 [00:00<00:00, 7510.75it/s]


In [ ]:
reward = run_policy_verbose(small_policy, verbose_level=1)

Current Total: 20
Current hand: King of Clubs, 10 of Spades
Dealer Total: 3
Dealer hand: 3 of Clubs
Learner takes action: HIT
Actions tested in training: []
Current Total: 30
Current hand: King of Clubs, 10 of Spades, Queen of Clubs
Dealer Total: 3
Dealer hand: 3 of Clubs
Player loses!


## Evaluating Q-learning algorithm.
If performance is worse than -0.07, then you probably have a bug in an earlier question. If your performance is better than -0.05, then you also probably have a bug in an earlier question.

In [ ]:
# let's train for 200000 games of blackjack. If a real game took about 5 minutes, this would be almost two years of continuous blackjack!
# This should not take very long
policy = run_Q_learning(200000, 0.05)

100%|██████████| 200000/200000 [00:13<00:00, 14479.88it/s]


In [ ]:
# Testing code:
def run_policy(learner):
  sim = BlackjackSimulator()
  while not sim.is_game_over():
    action = learner.get_action(sim)
    state = get_state(sim)
    sim.take_action(action)

  return sim.reward



def test_policy(policy, iterations=10000):
  '''
  tests a policy for `iterations` number of games.
  Returns the average reward obtained by the policy.
  '''
  pbar = tqdm(range(iterations))
  total_reward = 0.0
  for i in pbar:
    total_reward += run_policy(policy)

  return total_reward/iterations


Run this final cell to test implementation.

In [ ]:
# Test your learned policy. You should obtain a score of at least -0.07, and at most -0.05
# The house always wins eventually :)

test_policy(policy)

100%|██████████| 10000/10000 [00:00<00:00, 23152.84it/s]


-0.0578